In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import FeatureHasher

## Carrega dados

In [2]:
df_auditoria = pd.read_csv("ips-processed.csv")
df_auditoria.drop(columns=['Unnamed: 0'],inplace=True)
print(df_auditoria.shape)
df_auditoria.head()

(1000000, 4)


,DATA EVENTO,CPF,STATUS,REMOTE ADDRESS
0,2019-07-10 16:06:45,34894024407,SUCCESS,192.31.221.248
1,2019-07-10 16:06:49,34402518252,SUCCESS,192.56.54.69
2,2019-07-10 16:06:51,34641345139,SUCCESS,192.30.180.55
3,2019-07-10 16:06:46,34578456840,SUCCESS,192.49.255.9
4,2019-07-10 16:06:46,34808234821,SUCCESS,192.67.0.200


## cria outliers no dataset

In [3]:
df_auditoria['CPF2'] = df_auditoria['CPF'].apply(lambda x: 34730696066 if (x == 34235325539 or x== 34366617610 or x== 34892825288) else x)
df_auditoria['CPF'] = df_auditoria['CPF2']
df_auditoria.drop(columns='CPF2',inplace=True)
df_auditoria.CPF.value_counts()[:15]

34730696066    1375
34701401144     246
34420781073     225
34833549352     210
34744469440     201
34565540550     178
34462776270     173
34608136819     171
34572553252     171
34278037806     170
34338221820     170
34801902190     168
34458461841     161
34008360836     160
34702658267     159
Name: CPF, dtype: int64

In [4]:
df_auditoria['REMOTE ADDRESS2'] = df_auditoria['REMOTE ADDRESS'].apply(
    lambda x: '198.51.101.202' if (x == '192.0.3.40' or x== '198.51.101.220' 
    or x== '192.0.3.87' or x=='192.31.197.103' or x=='192.0.3.112' or x=='192.52.192.223'
    or x=='192.52.192.165' or x=='203.0.112.2' or x=='192.31.197.220' or x=='198.51.101.33') else x)
df_auditoria['REMOTE ADDRESS'] = df_auditoria['REMOTE ADDRESS2']
df_auditoria.drop(columns='REMOTE ADDRESS2',inplace=True)
df_auditoria['REMOTE ADDRESS'].value_counts()[:15]

198.51.101.202    575
192.52.192.227     50
192.0.3.24         50
192.0.3.123        50
198.51.101.105     50
198.51.101.118     50
192.52.192.46      50
192.31.197.56      49
192.31.197.159     49
203.0.112.41       49
198.51.101.63      49
192.52.192.124     49
192.0.1.130        49
192.0.1.226        49
198.51.101.86      49
Name: REMOTE ADDRESS, dtype: int64

In [5]:
def transform_ip(ip):
    """
    If IPv4, equalizes each group and left zero pads to match IPv6 length
    If IPv6, converts all to lower case
    """
    # IPv4 address
    groups = ip.split( "." )

    equalize_group_length = "".join( map( lambda group: group.zfill(3), groups ))
    return equalize_group_length

In [6]:
df_auditoria['IP'] = df_auditoria['REMOTE ADDRESS'].apply(transform_ip)
df_auditoria.head()

,DATA EVENTO,CPF,STATUS,REMOTE ADDRESS,IP
0,2019-07-10 16:06:45,34894024407,SUCCESS,192.31.221.248,192031221248
1,2019-07-10 16:06:49,34402518252,SUCCESS,192.56.54.69,192056054069
2,2019-07-10 16:06:51,34641345139,SUCCESS,192.30.180.55,192030180055
3,2019-07-10 16:06:46,34578456840,SUCCESS,192.49.255.9,192049255009
4,2019-07-10 16:06:46,34808234821,SUCCESS,192.67.0.200,192067000200


In [7]:
h = FeatureHasher(n_features=10, input_type='string')

In [8]:
transformed = h.transform( df_auditoria[['IP']].values.ravel()).toarray()
df = pd.DataFrame(transformed,columns=['IP1','IP2','IP3','IP4','IP5','IP6','IP7','IP8','IP9','IP10'])
df.head()

,IP1,IP2,IP3,IP4,IP5,IP6,IP7,IP8,IP9,IP10
0,1.0,5.0,-1.0,-1.0,0.0,0.0,0.0,-3.0,0.0,-1.0
1,4.0,3.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-3.0
2,3.0,2.0,0.0,-1.0,0.0,0.0,0.0,-2.0,0.0,-4.0
3,2.0,5.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-3.0
4,1.0,3.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,-6.0


In [9]:
df_auditoria = df_auditoria.merge(df,left_index=True, right_index=True)

In [10]:
df_auditoria[df_auditoria.IP=='198051101202']

,DATA EVENTO,CPF,STATUS,REMOTE ADDRESS,IP,IP1,IP2,IP3,IP4,IP5,IP6,IP7,IP8,IP9,IP10
1702,2019-07-10 16:09:04,34573111310,SUCCESS,198.51.101.202,198051101202,1.0,3.0,0.0,-1.0,0.0,0.0,0.0,-4.0,0.0,-3.0
4338,2019-07-10 16:14:39,34076485640,SUCCESS,198.51.101.202,198051101202,1.0,3.0,0.0,-1.0,0.0,0.0,0.0,-4.0,0.0,-3.0
7603,2019-07-10 16:21:39,34819363933,SUCCESS,198.51.101.202,198051101202,1.0,3.0,0.0,-1.0,0.0,0.0,0.0,-4.0,0.0,-3.0
7618,2019-07-10 16:22:29,34061054108,SUCCESS,198.51.101.202,198051101202,1.0,3.0,0.0,-1.0,0.0,0.0,0.0,-4.0,0.0,-3.0
7934,2019-07-10 16:22:35,34757569750,SUCCESS,198.51.101.202,198051101202,1.0,3.0,0.0,-1.0,0.0,0.0,0.0,-4.0,0.0,-3.0
12564,2019-07-10 16:33:30,34257537759,SUCCESS,198.51.101.202,198051101202,1.0,3.0,0.0,-1.0,0.0,0.0,0.0,-4.0,0.0,-3.0
15531,2019-07-10 16:39:40,34886474643,SUCCESS,198.51.101.202,198051101202,1.0,3.0,0.0,-1.0,0.0,0.0,0.0,-4.0,0.0,-3.0
16925,2019-07-10 16:43:10,34638511850,SUCCESS,198.51.101.202,198051101202,1.0,3.0,0.0,-1.0,0.0,0.0,0.0,-4.0,0.0,-3.0
17594,2019-07-10 16:44:35,34428234821,SUCCESS,198.51.101.202,198051101202,1.0,3.0,0.0,-1.0,0.0,0.0,0.0,-4.0,0.0,-3.0
19891,2019-07-10 16:50:17,34838538854,SUCCESS,198.51.101.202,198051101202,1.0,3.0,0.0,-1.0,0.0,0.0,0.0,-4.0,0.0,-3.0


In [11]:
df_auditoria.CPF.nunique()

327338

In [12]:
df_auditoria.head()

,DATA EVENTO,CPF,STATUS,REMOTE ADDRESS,IP,IP1,IP2,IP3,IP4,IP5,IP6,IP7,IP8,IP9,IP10
0,2019-07-10 16:06:45,34894024407,SUCCESS,192.31.221.248,192031221248,1.0,5.0,-1.0,-1.0,0.0,0.0,0.0,-3.0,0.0,-1.0
1,2019-07-10 16:06:49,34402518252,SUCCESS,192.56.54.69,192056054069,4.0,3.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-3.0
2,2019-07-10 16:06:51,34641345139,SUCCESS,192.30.180.55,192030180055,3.0,2.0,0.0,-1.0,0.0,0.0,0.0,-2.0,0.0,-4.0
3,2019-07-10 16:06:46,34578456840,SUCCESS,192.49.255.9,192049255009,2.0,5.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-3.0
4,2019-07-10 16:06:46,34808234821,SUCCESS,192.67.0.200,192067000200,1.0,3.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,-6.0
